### Extract points and apply transformations

In [ ]:
import svgpathtools
import numpy as np
from svgwrite import Drawing
from svgpathtools import parse_path
def apply_transformations(points, transform):
    if transform is None:
        return points

    if "translate" in transform:
        values = transform.split("translate(")[1].split(")")[0].split(",")
        dx, dy = float(values[0]), float(values[1]) if len(values) > 1 else 0
        points += np.array([dx, dy])

    if "scale" in transform:
        values = transform.split("scale(")[1].split(")")[0].split(",")
        sx, sy = float(values[0]), float(values[1]) if len(values) > 1 else float(values[0])
        points *= np.array([sx, sy])

    if "rotate" in transform:
        values = transform.split("rotate(")[1].split(")")[0].split()
        angle = np.radians(float(values[0]))
        cx, cy = float(values[1]), float(values[2]) if len(values) > 2 else (0, 0)
        rotation_matrix = np.array([[np.cos(angle), -np.sin(angle)], [np.sin(angle), np.cos(angle)]])
        points -= np.array([cx, cy])
        points = np.dot(points, rotation_matrix)
        points += np.array([cx, cy])

    if "skewX" in transform:
        angle = np.radians(float(transform.split("skewX(")[1].split(")")[0]))
        skew_matrix = np.array([[1, np.tan(angle)], [0, 1]])
        points = np.dot(points, skew_matrix)

    if "skewY" in transform:
        angle = np.radians(float(transform.split("skewY(")[1].split(")")[0]))
        skew_matrix = np.array([[1, 0], [np.tan(angle), 1]])
        points = np.dot(points, skew_matrix)

    return points

def extract_and_store_svg(input_file):
    svg_elements, attributes, _ = svgpathtools.svg2paths2(input_file)
    stored_data = []

    for path, attribute in zip(svg_elements, attributes):
        element_type = attribute.get('element')
        transform = attribute.get('transform')
        # print(f"Attributes: {attribute}")  # Debugging: print attributes
        element_type = attribute.get('element', 'path')  # Default to 'path' if not specified
        transform = attribute.get('transform')
        # print(f"Element Type: {element_type}")  # Debugging: print element type

        if element_type == 'rect':
            x = float(attribute['x'])
            y = float(attribute['y'])
            width = float(attribute['width'])
            height = float(attribute['height'])
            points = np.array([[x, y], [x + width, y], [x + width, y + height], [x, y + height]])
            points = apply_transformations(points, transform)
            stored_data.append(points.tolist())

        elif element_type == 'circle':
            cx = float(attribute['cx'])
            cy = float(attribute['cy'])
            r = float(attribute['r'])
            center = np.array([cx, cy])
            center = apply_transformations(center, transform)
            stored_data.append({'center': center.tolist(), 'radius': r})

        elif element_type == 'ellipse':
            cx = float(attribute['cx'])
            cy = float(attribute['cy'])
            rx = float(attribute['rx'])
            ry = float(attribute['ry'])
            center = np.array([cx, cy])
            center = apply_transformations(center, transform)
            stored_data.append({'center': center.tolist(), 'rx': rx, 'ry': ry})

        elif element_type in ['line', 'polyline', 'polygon']:
            points = np.array([[float(coord.split(',')[0]), float(coord.split(',')[1])] for coord in attribute['points'].split()])
            points = apply_transformations(points, transform)
            stored_data.append(points.tolist())
          
        elif element_type == 'path':
            path_data = parse_path(attribute['d'])
            points = []

            for segment in path_data:
                if isinstance(segment, svgpathtools.Line):
                    points.append([segment.start.real, segment.start.imag])
                    points.append([segment.end.real, segment.end.imag])
                elif isinstance(segment, svgpathtools.CubicBezier):
                    # Sample points along the cubic bezier curve
                    for t in np.linspace(0, 1, num=100):
                        point = segment.point(t)
                        points.append([point.real, point.imag])
                elif isinstance(segment, svgpathtools.QuadraticBezier):
                    # Sample points along the quadratic bezier curve
                    for t in np.linspace(0, 1, num=100):
                        point = segment.point(t)
                        points.append([point.real, point.imag])
                elif isinstance(segment, svgpathtools.Arc):
                    # Sample points along the arc
                    for t in np.linspace(0, 1, num=100):
                        point = segment.point(t)
                        points.append([point.real, point.imag])

            if points:  # Only store if points were found
                points = np.array(points)
                if transform:
                    points = apply_transformations(points, transform)
                stored_data.append(points.tolist())
            else:
                print("No points found for this path.")  # Debugging: No points found

    return stored_data

# Example usage:
input_svg_file = r"problems\problems\isolated.svg"
stored_data = extract_and_store_svg(input_svg_file)
print(len(stored_data))
print(len(stored_data[0]))
print(len(stored_data[0][0]))

### Find the intersections between lines and segments accordingly and store their sub-parts

In [ ]:
import numpy as np

def line_equation(p1, p2):
    """Returns the slope (m) and intercept (b) of a line given two points."""
    if p2[0] - p1[0] == 0:  # Vertical line
        return None, p1[0]
    m = (p2[1] - p1[1]) / (p2[0] - p1[0])
    b = p1[1] - m * p1[0]
    return m, b

def find_intersection(line1, line2):
    """Find the intersection of two lines, if it exists."""
    m1, b1 = line1
    m2, b2 = line2
    
    if m1 is None:  # First line is vertical
        x_intersect = b1
        y_intersect = m2 * x_intersect + b2
    elif m2 is None:  # Second line is vertical
        x_intersect = b2
        y_intersect = m1 * x_intersect + b1
    else:
        if m1 == m2:
            return None  # Parallel lines
        x_intersect = (b2 - b1) / (m1 - m2)
        y_intersect = m1 * x_intersect + b1
    
    return x_intersect, y_intersect

def on_segment(p, q, r):
    """Check if point q lies on line segment pr."""
    return (q[0] <= max(p[0], r[0]) and q[0] >= min(p[0], r[0]) and
            q[1] <= max(p[1], r[1]) and q[1] >= min(p[1], r[1]))

def segment_intersects(p1, p2, p3, p4):
    """Check if line segment p1p2 intersects with p3p4."""
    line1 = line_equation(p1, p2)
    line2 = line_equation(p3, p4)
    
    if line1[0] == line2[0]:  # Parallel or Collinear lines
        if line1[1] == line2[1] and on_segment(p1, p3, p2):
            return p3  # One of the endpoints of the overlapping segment
        return None  # Parallel and non-overlapping
    
    intersection = find_intersection(line1, line2)
    
    if intersection is None:
        return None
    
    if (on_segment(p1, intersection, p2) and
        on_segment(p3, intersection, p4)):
        return intersection
    
    return None

def smooth_points(points, factor=0.5):
    """Apply basic smoothing to a list of points."""
    smoothed_points = []
    for i in range(1, len(points) - 1):
        prev_point = np.array(points[i - 1])
        curr_point = np.array(points[i])
        next_point = np.array(points[i + 1])
        
        new_point = curr_point + factor * (next_point + prev_point - 2 * curr_point)
        smoothed_points.append(new_point.tolist())
    return [points[0]] + smoothed_points + [points[-1]]

def analyze_intersections_in_set(segment_set):
    """Analyze intersections within a single set of segments."""
    processed_segments = []
    segment_seen = set()
    
    num_points = len(segment_set)
    
    for i in range(0, num_points - 1):
        for j in range(i + 1, num_points - 1):
            p1, p2 = segment_set[i], segment_set[i + 1]
            p3, p4 = segment_set[j], segment_set[j + 1]
            
            # Convert segments to tuples so they can be stored in the set
            segment1 = (tuple(p1), tuple(p2))
            segment2 = (tuple(p3), tuple(p4))
            
            if (segment1, segment2) in segment_seen or (segment2, segment1) in segment_seen:
                continue
            
            intersection = segment_intersects(p1, p2, p3, p4)
            
            if intersection:
                smoothed_intersection = smooth_points([p1, intersection, p2])
                processed_segments.append(smoothed_intersection)
                
                smoothed_intersection = smooth_points([p3, intersection, p4])
                processed_segments.append(smoothed_intersection)
                
                segment_seen.add((segment1, segment2))
            else:
                processed_segments.append([p1, p2])
                # Properly create new segments with adjusted positions]
                processed_segments.append([p3, p4])
                
    return processed_segments

# Assuming `stored_data` is provided and correctly initialized
finalres = []
for idx, segment_set in enumerate(stored_data):
    print(f"Processing set {idx + 1}/{len(stored_data)}")
    resulting_segments = analyze_intersections_in_set(segment_set)
    for seg in resulting_segments:
        finalres.append(seg)


In [ ]:
print(len(finalres))

### Use cycle detection in graph to extract connected shapes

In [ ]:
import numpy as np
from collections import defaultdict

class Graph:
    def __init__(self):
        self.graph = defaultdict(list)
        self.cycles = []

    def add_edge(self, u, v):
        # Add the edge between two unique integer points
        self.graph[u].append(v)
        self.graph[v].append(u)

    def dfs(self, v, visited, parent, path):
        visited[v] = True
        path.append(v)

        for neighbor in self.graph[v]:
            if not visited[neighbor]:
                self.dfs(neighbor, visited, v, path)
            elif neighbor != parent:
                if neighbor in path:
                    cycle_start_index = path.index(neighbor)
                    cycle = path[cycle_start_index:] + [neighbor]
                    # Remove duplicates while preserving order
                    unique_cycle = []
                    [unique_cycle.append(point) for point in cycle if point not in unique_cycle]
                    if unique_cycle not in self.cycles and len(unique_cycle) > 1:
                        self.cycles.append(unique_cycle)

        path.pop()

    def find_cycles(self):
        visited = {node: False for node in self.graph}
        for node in self.graph:
            if not visited[node]:
                self.dfs(node, visited, None, [])
        return self.cycles
#convert each point to integer and remove duplicates
def convert_to_unique_integer_points(segments):
    """
    Convert all points by first adding and subtracting 0.05, 
    then round them to integers, include the original points,
    and remove duplicates.
    """
    unique_points = set()
    int_segments = set()

    for seg in segments:
        # Original points
        p1 = tuple(int(round(coord)) for coord in seg[0])
        p2 = tuple(int(round(coord)) for coord in seg[1])

        # Points with 0.05 added/subtracted
        p1_add = tuple(int(round(coord + 0.05)) for coord in seg[0])
        p1_sub = tuple(int(round(coord - 0.05)) for coord in seg[0])
        p2_add = tuple(int(round(coord + 0.05)) for coord in seg[1])
        p2_sub = tuple(int(round(coord - 0.05)) for coord in seg[1])
        
        # Update unique points set
        unique_points.update([p1, p2, p1_add, p1_sub, p2_add, p2_sub])
        
        # Update integer segments set
        int_segments.update([
            tuple([p1, p2])
        ])

    return int_segments, unique_points

# Function to create a graph from integer segments
def create_graph_from_segments(segments):
    graph = Graph()
    for seg in segments:
        p1, p2 = seg[0], seg[1]
        graph.add_edge(p1, p2)
    return graph

# Convert finalres to integer points and create the graph
int_segments, unique_points = convert_to_unique_integer_points(finalres)
graph = create_graph_from_segments(int_segments)
cycles = graph.find_cycles()

# Output the found cycles
print("Detected cycles in the graph representation:")
print(len(cycles))
for cycle in cycles:
    print(cycle)


In [ ]:
cycles

### Plot the obtained cycles

In [ ]:
import matplotlib.pyplot as plt

def plot_cycle(cycle_points, title="Cycle Plot"):
    """
    Plots a single cycle of points.

    Parameters:
    - cycle_points: List of points in the cycle, each point is a tuple or list (x, y).
    - title: Title of the plot.
    """
    # Unzip the list of points into x and y coordinates
    x_coords, y_coords = zip(*cycle_points)
    
    # Add the first point at the end to close the cycle
    x_coords += (x_coords[0],)
    y_coords += (y_coords[0],)
    
    # Plot the points and the connecting lines
    plt.figure(figsize=(6, 6))
    plt.plot(x_coords, y_coords, marker='o', linestyle='-', color='blue')
    
    # Mark the points with their coordinates for clarity
    for i, (x, y) in enumerate(cycle_points):
        plt.text(x, y, f'({x}, {y})', fontsize=9, ha='right')
    
    plt.title(title)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.grid(True)
    plt.gca().set_aspect('equal', adjustable='box')
    plt.show()

for cycle in cycles:
    plot_cycle(cycle, title="Example Cycle Plot")


### Implement
#### Clustering: Grouping nearby points.
#### Polygon Approximation: Simplifying the polygon using the Polygon.simplify() method.
#### Convex Hull Check: Ensuring the polygon is convex and applying the convex hull if it is.
#### RDP Simplification: Further simplifying the shape using the Ramer-Douglas-Peucker algorithm.
#### Collinearity Reduction: Reducing collinear points.
#### Curvature Calculation: Calculating curvature for non-uniform smoothing.
#### Non-uniform Smoothing: Applying Gaussian smoothing based on curvature.
#### Fourier Transform Smoothing: Applying Fourier transform to remove high-frequency noise.

In [ ]:
from sklearn.cluster import DBSCAN
import numpy as np
from scipy.spatial import ConvexHull, distance
from scipy.ndimage import gaussian_filter1d
from scipy.fft import fft, ifft
from shapely.geometry import Polygon

def euclidean_distance(p, q):
    return np.sqrt((p[0] - q[0])**2 + (p[1] - q[1])**2)

def approximate_polygon(points, tolerance=3):
    polygon = Polygon(points)
    simplified_polygon = polygon.simplify(tolerance, preserve_topology=True)
    return list(simplified_polygon.exterior.coords)

def convex_hull_approximation(points):
    hull = ConvexHull(points)
    return [points[i] for i in hull.vertices]

def rdp(points, epsilon=1.0):
    if len(points) < 3:
        return points

    start, end = points[0], points[-1]
    max_dist = 0
    index = 0
    for i in range(1, len(points) - 1):
        dist = distance.euclidean(points[i], start) + distance.euclidean(points[i], end)
        if dist > max_dist:
            index, max_dist = i, dist

    if max_dist > epsilon:
        left = rdp(points[:index + 1], epsilon)
        right = rdp(points[index:], epsilon)
        return left[:-1] + right
    else:
        return [start, end]

def is_convex(points):
    points_np = np.array(points)
    hull = ConvexHull(points_np)
    hull_points = set(map(tuple, points_np[hull.vertices]))
    original_points = set(map(tuple, points_np))
    return hull_points == original_points

def adaptive_collinearity(A, B, C, threshold_ratio=0.1, range_offset=2):
    A = (A[0] + range_offset, A[1] + range_offset)
    B = (B[0] + range_offset, B[1] + range_offset)
    C = (C[0] + range_offset, C[1] + range_offset)
    
    area = abs(A[0] * (B[1] - C[1]) + B[0] * (C[1] - A[1]) + C[0] * (A[1] - B[1]))
    max_len = max(euclidean_distance(A, B), euclidean_distance(B, C), euclidean_distance(A, C))
    
    return area < (threshold_ratio * max_len) + range_offset

def collinearpoints(points):
    if len(points) < 3:
        return points
    
    result = []
    i = 0
    while i < len(points) - 2:
        A = points[i]
        B = points[i + 1]
        C = points[i + 2]
        
        if adaptive_collinearity(A, B, C):
            result.append(A)
            while i < len(points) - 2 and adaptive_collinearity(A, points[i + 1], points[i + 2]):
                i += 1
            result.append(points[i + 1])
        else:
            result.append(A)
        i += 1
    
    if i < len(points):
        result.append(points[-1])
    
    return result

# Fourier smoothing function
def fourier_smoothing(points, cutoff_frequency=0.1):
    x_coords = np.array([p[0] for p in points])
    y_coords = np.array([p[1] for p in points])
    
    # Apply Fourier Transform
    x_fft = fft(x_coords)
    y_fft = fft(y_coords)
    
    # Create frequency array
    frequencies = np.fft.fftfreq(len(points))
    
    # Filter frequencies
    x_fft[np.abs(frequencies) > cutoff_frequency] = 0
    y_fft[np.abs(frequencies) > cutoff_frequency] = 0
    
    # Inverse Fourier Transform
    smoothed_x = ifft(x_fft).real
    smoothed_y = ifft(y_fft).real
    
    smoothed_points = list(zip(smoothed_x, smoothed_y))
    return smoothed_points

# Function to calculate curvature at each point
def calculate_curvature(points):
    curvatures = []
    for i in range(1, len(points) - 1):
        A, B, C = points[i - 1], points[i], points[i + 1]
        ab = np.linalg.norm(np.array(B) - np.array(A))
        bc = np.linalg.norm(np.array(C) - np.array(B))
        ac = np.linalg.norm(np.array(C) - np.array(A))
        if ab * bc == 0:
            curvature = 0
        else:
            curvature = abs(np.arccos((ab**2 + bc**2 - ac**2) / (2 * ab * bc)))
        curvatures.append(curvature)
    
    # Pad curvatures to match the length of points
    if curvatures:
        curvatures.insert(0, curvatures[0])
        curvatures.append(curvatures[-1])
    
    return curvatures

# Non-uniform smoothing based on curvature
def non_uniform_smoothing(points, curvatures, min_sigma=0.5, max_sigma=2.0):
    smoothed_points = []
    for i in range(len(points)):
        sigma = np.interp(curvatures[i], [min(curvatures), max(curvatures)], [min_sigma, max_sigma])
        smoothed_x = gaussian_filter1d([p[0] for p in points], sigma=sigma)
        smoothed_y = gaussian_filter1d([p[1] for p in points], sigma=sigma)
        smoothed_points.append((smoothed_x[i], smoothed_y[i]))
    return smoothed_points

# Function to classify geometric shapes
simplified_cycle = []

for cycle in cycles:
    snapped_points = cycle
    
    # Step 1: Calculate curvature for non-uniform smoothing
    curvatures = calculate_curvature(snapped_points)
    
    # Step 2: Apply non-uniform smoothing
    non_uniform_smoothed_points = non_uniform_smoothing(snapped_points, curvatures)
    
    # Step 3: Apply Fourier smoothing
    fourier_smoothed_points = fourier_smoothing(non_uniform_smoothed_points, cutoff_frequency=0.2)
    
    # Step 4: Approximate the polygon
    simplified_points = approximate_polygon(fourier_smoothed_points, tolerance=0.5)
    
    # Step 5: Check if the shape is convex and apply convex hull if necessary
    if not is_convex(simplified_points):
        simplified_points = convex_hull_approximation(simplified_points)
    
    # Step 6: Further simplify using RDP
    simplified_points = rdp(simplified_points, epsilon=1.0)
    
    # Step 7: Reduce collinear points
    collinear_reduced_points = collinearpoints(simplified_points)
    
    # Step 8: Round the final points to integer coordinates
    integer_points = [(round(p[0]), round(p[1])) for p in collinear_reduced_points]
    
    # Store the final simplified cycle with integer coordinates
    simplified_cycle.append(integer_points)
result=simplified_cycle
# simplified_cycle now contains the fully simplified and smoothed versions of the cycles.


### Once the points have been simplified, compute side lengths, angles, centroids, and other relevant metrics. Use mathematical formulas, least squares methods, ellipse fitting, or distance calculations from the center to estimate the shape.

In [ ]:
import math
import numpy as np
from sklearn.cluster import DBSCAN
from scipy.optimize import curve_fit

# Calculate Euclidean distance between two points
def euclidean_distance(p, q):
    return np.sqrt((p[0] - q[0])**2 + (p[1] - q[1])**2)

# Calculate angle between three points A, B, C using cosine rule
def calculate_angle(A, B, C):
    AB = euclidean_distance(A, B)
    BC = euclidean_distance(B, C)
    AC = euclidean_distance(A, C)
    if AB == 0 or BC == 0:  # Prevent division by zero
        return 0
    cos_value = max(-1, min(1, (AB**2 + BC**2 - AC**2) / (2 * AB * BC)))
    return math.degrees(math.acos(cos_value))

# Remove duplicate points by averaging close points
def remove_duplicate_points(points, distance_threshold=2):
    if not isinstance(points, (list, tuple)):
        raise TypeError(f"Expected a list or tuple of points, but got {type(points)}")
    points = np.array(points)
    cleaned_points = []
    used_indices = set()

    for i in range(len(points)):
        if i in used_indices:
            continue
        
        p = points[i]
        distances = np.linalg.norm(points - p, axis=1)
        close_indices = np.where(distances <= distance_threshold)[0]
        
        if len(close_indices) > 1:
            avg_point = np.mean(points[close_indices], axis=0)
            cleaned_points.append(tuple(avg_point))
            used_indices.update(close_indices)
        else:
            cleaned_points.append(tuple(p))
            used_indices.add(i)
    
    return cleaned_points

# Calculate the centroid of a polygon
def calculate_centroid(points):
    points = np.array(points)
    centroid = np.mean(points, axis=0)
    return tuple(centroid)

# Calculate side lengths of the polygon
def calculate_side_lengths(points):
    return [
        euclidean_distance(points[i], points[(i + 1) % len(points)])
        for i in range(len(points))
    ]

# Calculate interior angles of the polygon
def calculate_interior_angles(points):
    return [
        calculate_angle(points[i-1], points[i], points[(i+1) % len(points)])
        for i in range(len(points))
    ]

# Calculate distances from each point to the centroid
def calculate_distances_from_centroid(points, centroid):
    return np.linalg.norm(np.array(points) - np.array(centroid), axis=1)

# Check if the points approximately form a circle
def check_circle(points, centroid, tolerance=0.1):
    distances = calculate_distances_from_centroid(points, centroid)
    avg_distance = np.mean(distances)
    return np.sum(np.abs(distances - avg_distance) <= tolerance) >= len(points) / 2

# Function to fit a line segment to points using least squares
def fit_line_least_squares(points):
    x = np.array([p[0] for p in points])
    y = np.array([p[1] for p in points])
    
    # Fit a line (degree=1 for linear)
    m, c = np.polyfit(x, y, 1)
    return m, c

# Calculate residuals of the points with respect to the line y = mx + c
def calculate_residuals(points, m, c):
    x = np.array([p[0] for p in points])
    y = np.array([p[1] for p in points])
    
    y_pred = m * x + c
    residuals = np.abs(y - y_pred)
    return residuals

# Adaptive thresholding for fitting line segments
def adaptive_thresholding(points, base_threshold=2.5, increase_factor=1.2, max_attempts=5):
    attempts = 0
    threshold = base_threshold

    while attempts < max_attempts:
        m, c = fit_line_least_squares(points)
        residuals = calculate_residuals(points, m, c)
        mean_residual = np.mean(residuals)

        if mean_residual < threshold:
            return m, c, residuals

        threshold *= increase_factor
        attempts += 1

    return m, c, residuals  # Final attempt results

# Cluster points using DBSCAN
def cluster_points(points, epsilon=1.0):
    db = DBSCAN(eps=epsilon, min_samples=1).fit(points)
    labels = db.labels_
    clusters = {}
    for label in set(labels):
        clusters[label] = np.array([points[i] for i in range(len(points)) if labels[i] == label])
    return clusters

# Process clusters to fit line segments and classify based on residuals
def process_clusters(clusters):
    line_segments = []
    remaining_points = []
    
    for label, cluster_points in clusters.items():
        if len(cluster_points) < 2:
            remaining_points.extend(cluster_points)
            continue
        
        m, c, residuals = adaptive_thresholding(cluster_points)
        mean_residual = np.mean(residuals)
        
        final_threshold = 5.0  # Adjust as needed
        if mean_residual < final_threshold:
            line_segments.append((m, c))
        else:
            remaining_points.extend(cluster_points)
    
    return line_segments, remaining_points
# Ellipse equation
def ellipse_equation(x, h, k, a, b):
    """Ellipse equation in terms of x."""
    return k + b * np.sqrt(1 - ((x - h) ** 2) / a ** 2)

# Fit ellipse to points
def fit_ellipse(points):
    x = points[:, 0]
    y = points[:, 1]
    initial_guess = [np.mean(x), np.mean(y), np.std(x), np.std(y)]
    params, _ = curve_fit(lambda x, h, k, a, b: ellipse_equation(x, h, k, a, b), x, y, p0=initial_guess)
    return params

# Check if points fit an ellipse
def is_ellipse(points, tolerance=2.0):
    params = fit_ellipse(points)
    x = points[:, 0]
    y = points[:, 1]
    residuals = np.abs(y - ellipse_equation(x, *params))
    return np.all(residuals < tolerance), params

# Classify based on line segments
def classify_based_on_lines(line_segments, centroid, remaining_points):
    num_lines = len(line_segments)
    shape = "Unknown"
    if num_lines == 1:
        shape = "Line"
    elif num_lines == 4:
        side_lengths = []
        for m, c in line_segments:
            for i, (m1, c1) in enumerate(line_segments):
                for j, (m2, c2) in enumerate(line_segments):
                    if i < j:
                        x_intersect = (c2 - c1) / (m1 - m2) if m1 != m2 else 0
                        y_intersect = m1 * x_intersect + c1
                        side_lengths.append(euclidean_distance((0, c1), (x_intersect, y_intersect)))

        side_lengths = sorted(side_lengths)
        if len(side_lengths) >= 4:
            if all(abs(side_lengths[i] - side_lengths[(i+1) % 4]) < 2 for i in range(0, 4, 2)):
                shape = "Square"
            else:
                shape = "Rectangle"
    
    elif num_lines > 4:
        distances = [euclidean_distance(line_segments[i][0], line_segments[(i+1) % num_lines][0])
                     for i in range(num_lines)]
        angles = calculate_angle([segment[0] for segment in line_segments])

        if all(abs(length - distances[0]) < 1e-5 for length in distances):
            if all(abs(angle - angles[0]) < 1e-5 for angle in angles):
                shape = "Regular Polygon"
            else:
                shape = "Regular Star"
    elif num_lines == 3:
        shape = "Triangle"
    
    else:
        if len(remaining_points) > 20 and check_circle(remaining_points, centroid):
            shape = "Circle"
        elif is_ellipse(np.array(remaining_points))[0]:
            shape = "Ellipse"
        
    return shape, centroid

# Apply the classification to each cycle
def apply_to_cycles(cycles):
    simplified_cycles = []
    
    for cycle in cycles:
        if not cycle:
            continue
        integer_points = remove_duplicate_points(cycle)
        clusters = cluster_points(np.array(integer_points), epsilon=1.0)
        line_segments, remaining_points = process_clusters(clusters)
        centroid = calculate_centroid(integer_points)
        shape, centroid = classify_based_on_lines(line_segments, centroid, remaining_points)
        
        simplified_cycles.append({
            "line_segments": line_segments,
            "remaining_points": remaining_points,
            "shape": shape,
            "centroid": centroid
        })
    
    return simplified_cycles

# Process the result data
for points in result:
    points = remove_duplicate_points(points)

answer = apply_to_cycles(result)

# Print or use the results
for cycle in answer:
    print("Line Segments:")
    for segment in cycle["line_segments"]:
        print(f"Line: y = {segment[0]}x + {segment[1]}")
    
    print("Remaining Points:")
    for point in cycle["remaining_points"]:
        print(point)
    
    print(f"The shape is classified as: {cycle['shape']}")
    print(f"Centroid: {cycle['centroid']}")
    print()
